In [2]:
import sys
sys.path.append(r'C:/Program Files (zk)/PythonFiles/AClassification/AudioClassification-CoughVID')
import os
import yaml
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from models.conv_vae import ConvVAE

from readers.coughvid_reader import CoughVID_Dataset

In [3]:
import numpy as np
def CoughVID_NormalAnomaly(filename="./datasets/waveinfo_annotation.csv", istrain=True, isdemo=False):
    healthy_p_list = []
    unhealthy_p_list = []
    healthy_label_list = []
    unhealthy_label_list = []
    # healthy_anomaly_list = []
    # unhealthy_anomaly_list = []
    with open(filename, 'r') as fin:
        fin.readline()
        line = fin.readline()
        while line:
            parts = line.split(',')
            status = int(parts[2])
            if status == 0:
                healthy_p_list.append(parts[1])
                healthy_label_list.append(np.array(parts[2], dtype=np.int64))
                # healthy_anomaly_list.append(np.array(0, dtype=np.int64))
            else:
                
                unhealthy_p_list.append(parts[1])
                unhealthy_label_list.append(np.array(parts[2], dtype=np.int64))
                # unhealthy_anomaly_list.append(np.array(1, dtype=np.int64))
            line = fin.readline()
    return healthy_p_list, healthy_label_list, unhealthy_p_list, unhealthy_label_list

# print(len(healthy_anomaly_list))
# print(len(unhealthy_anomaly_list))

11297 3788
11297 3788


## 读取数据集

In [4]:
healthy_p_list, healthy_label_list, unhealthy_p_list, unhealthy_label_list = CoughVID_NormalAnomaly()
print(len(healthy_p_list), len(unhealthy_p_list))
print(len(healthy_label_list), len(unhealthy_label_list))

trp, trl, vap, val = healthy_p_list[:10400], healthy_label_list[:10400], healthy_p_list[10400:], healthy_label_list[10400:]
print(len(trp), len(trl))
print(len(vap), len(val))

10400 10400
897 897


## 最耗时的读取

In [8]:
tic = time.time()
train_dataset = CoughVID_Dataset(path_list=trp, label_list=trl)
toc = time.time()
print("Train Dataset Creat Completely, cost time:", toc-tic)
valid_dataset = CoughVID_Dataset(path_list=vap, label_list=val)
toc = time.time()
print("Valid Dataset Creat Completely, cost time:", toc-tic)

Loading:   0%|                                                                               | 0/11297 [00:00<?, ?it/s]C:\Program Files (zk)\PythonFiles\AClassification\AudioClassification-CoughVID\readers\audio.py:117: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, sample_rate = librosa.core.load(file)  # , dtype='float32')
C:\Users\zhaoke\miniconda3\envs\torch-0\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
Loading:   0%|                                                                     | 1/11297 [00:00<2:57:53,  1.06it/s]C:\Program Files (zk)\PythonFiles\AClassification\AudioClassification-CoughVID\readers\audio.py:117: UserWarning: PySoundFile failed. Trying audioread instead.
  samples, sample_rate = librosa.core.load(file)  # , dtype='float32')
C:\Users\zhao

AttributeError: 'av.audio.fifo.AudioFifo' object has no attribute '_samples'

## 基本参数配置

In [2]:
configs = {
    "run_save_dir": "./runs/tdnn_coughvid/",
    "model":{
        "num_class": 3,
        "input_length": 94,
        "wav_length": 48000,
        "input_dim": 512,
        "n_mels": 80,
        },
    "fit":{
        "batch_size": 64,
        "epochs" : 23,
        "start_scheduler_epoch": 6
        },
}

device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
num_epoch = configs["fit"]["epochs"]
klw = 0.00025
# istrain: 如果是评估环节，设为False，读取测试集，并且不创建optimizer
# isdemo: 如果只是测试一下，设为True，仅读取32条数据方便快速测试是否有bug
istrain, isdemo = True, False

In [3]:
# model loss_function optimizer scheduler
print("All model and loss are on device:", device)
model = ConvVAE(shape=(1, 94, 128), flat=False).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=5e-5)
print("Create TDNN, Adam with lr=1e-3, CosineAnnealingLR Shceduler")

MSE_loss = nn.MSELoss(reduction="mean")
def vae_loss(X, X_hat, mean, logvar, kl_weight=0.0001):
    reconstruction_loss = MSE_loss(X_hat, X)
    KL_divergence = 0.5 * torch.sum(-1 - logvar + torch.exp(logvar) + mean ** 2)
    # print(reconstruction_loss.item(), KL_divergence.item())
    return reconstruction_loss + kl_weight * KL_divergence

print("Create VAELoss...")

All model and loss are on device: cuda
Create TDNN, Adam with lr=1e-3, CosineAnnealingLR Shceduler
Create VAELoss...


In [ ]:
### 测试模型正确性

In [ ]:
# device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
# m = ConvVAE(shape=(1, 94, 128), flat=True).to(device)
# x = torch.randn(size=(16, 1, 94, 128)).to(device)

# # print(m(x).shape)
# recon_mel, z, mean_lant, logvar_lant = m(x)  # (28-3+2p)/2+1
# print(recon_mel.shape)
# print(mean_lant)

In [10]:
# 运行保存目录

In [11]:
timestr = time.strftime("%Y%m%d%H%M", time.localtime())
if istrain:
    run_save_dir = configs["run_save_dir"] + timestr + f'_tdnn_focalloss/'
if not isdemo:
    os.makedirs(run_save_dir, exist_ok=True)
    print("创建运行保存文件", run_save_dir)

创建运行保存文件 ./runs/tdnn_coughvid/202404251743_tdnn_focalloss/


In [ ]:
history1 = []
for epoch_id in range(configs["fit"]["epochs"]):
    # ---------------------------
    # -----------TRAIN-----------
    # ---------------------------
    model.train()
    for x_idx, (x_wav, y_label, _) in enumerate(tqdm(train_loader, desc="Training")):
        x_mel = w2m(x_wav).to(device)
        y_label = torch.tensor(y_label, device=device)
        # print("shape of x_mel:", x_mel.shape)
        optimizer.zero_grad()
        recon_mel, _, mean_lantet, logvar_latent = model(x=x_mel)
        # recon_loss = self.recon_loss(recon_spec, x_mel)
        pred_loss = vae_loss(x_mel, recon_mel, mean_latent, logvar_latent)
        pred_loss.backward()
        optimizer.step()

        if x_idx > 2:
            history1.append(pred_loss.item())
        if x_idx % 60 == 0:
            print(f"Epoch[{epoch_id}], mtid pred loss:{pred_loss.item():.4f}")
    if epoch_id >= configs["fit"]["start_scheduler_epoch"]:
        scheduler.step()

    # ---------------------------
    # -----------SAVE------------
    # ---------------------------
    plt.figure(0)
    plt.plot(range(len(history1)), history1, c="green", alpha=0.7)
    plt.savefig(run_save_dir + f'cls_loss_iter_{epoch_id}.png')
    plt.close()
    # if epoch > 6 and epoch % 2 == 0:
    os.makedirs(run_save_dir + f"model_epoch_{epoch_id}/", exist_ok=True)
    tmp_model_path = "{model}model_{epoch}.pth".format(
        model=run_save_dir + f"model_epoch_{epoch_id}/",
        epoch=epoch_id)
    torch.save(model.state_dict(), tmp_model_path)
    # ---------------------------
    # -----------TEST------------
    # ---------------------------
    model.eval()
    heatmap_input = None
    labels = None
    for x_idx, (x_wav, y_label, _) in enumerate(tqdm(valid_loader, desc="Validate")):
        x_mel = w2m(x_wav).to(device)
        y_label = torch.tensor(y_label, device=device)
        _, z, _, _ = model(x=x_mel)
        if x_idx == 0:
            heatmap_input, labels = y_pred, y_label
        else:
            heatmap_input = torch.concat((heatmap_input, y_pred), dim=0)
            labels = torch.concat((labels, y_label), dim=0)
        # if x_idx * configs["fit"]["batch_size"] > 800:
        #     break
    print("heatmap_input shape:", heatmap_input.shape)
    print("lables shape:", labels.shape)
    # if epoch > 3:
    #     self.plot_reduction(resume_path="", load_epoch=epoch, reducers=["heatmap"])
    heatmap_input = heatmap_input.detach().cpu().numpy()
    labels = labels.detach().cpu().numpy()
    calc_accuracy(pred_matrix=heatmap_input, label_vec=labels,
                  save_path=run_save_dir + f"/accuracy_epoch_{epoch_id}.txt")
    plot_heatmap(pred_matrix=heatmap_input, label_vec=labels,
                 ticks=["healthy", "symptomatic", "COVID-19"],
                 save_path=run_save_dir + f"/heatmap_epoch_{epoch_id}.png")
print("============== END TRAINING ==============")

In [ ]:
# 测试集读取和测试

In [ ]:
tep, tel = CoughVID_Lists(filename="./datasets/waveinfo_annotation.csv", istrain=False, isdemo=False)
print(len(tep), len(tel))

In [ ]:
tic = time.time()
test_dataset = CoughVID_Dataset(path_list=tep, label_list=tel)
toc = time.time()
print("Test Dataset Creat Completely, cost time:", toc-tic)

# End